# Train Body Part Regression Model 
we will exemplary show how to train a Body Part Regression Model. In general it is important to create a very diverse dataset for training with lots of CT volumes (preferable over 1000 volumes) and from several different studies. For demonstration purposes, we will use in this notebook fewer data and only from one study. 

## 1. Download data 
First, the data needs to be downloaded. We will use the [CT Lymph Nodes Dataset](https://wiki.cancerimagingarchive.net/display/Public/CT+Lymph+Nodes) from the TCIA. <br> 
Select the dataset from the [search porta](https://nbia.cancerimagingarchive.net/nbia-search/) and download the data. This will take some minutes. 

![title](data/tcia-download-data.png)

For evaluation purposes, a corresponding landmark file is needed. 
First, we need to define landmarks, on which we want to evaluate the BPR model. Bone landmarks are more robust than organ landmarks and therefore more suitable. In this example, we will use the landmarks: pelvis-start, femur-end, L5, L3, L1, Th11, Th8, Th5, and Th2. In general, the centroid of the spines work well as evaluation landmarks. The spine landmarks can get automatically annotated with a vertebra [nnU-Net](https://github.com/MIC-DKFZ/nnUNet). 
For this example a subset of the data has been annotated already and can be found in the excel file: <br>
*data/ct-lymph-nodes-annoted-landmarks.xlsx*  <br> 
For the sake of simplicity, we have in this example only a test and training (and no validation set). 


## 2. Preprocess data 
After the data has downloaded the data needs to be preprocessed to npy-arrays. In this example, we will downscale the axial slices to a size of 64 x64

In [ ]:
nifti_to_npy() 